# First Data Preprocessing

In [1]:
# Built-in packages
import string
import os
import time as t
import warnings
import sys

# Basic Packages for Data Wrangling
import spacy
import pymysql
import pandas as pd
import numpy as np
from numpy import array
import datetime as dt
import pycountry as pc # Country Name Converter
import re # For regular expression

# Handmade packages fro data preprocessing
from tool_functions import remove_short_words, remove_punctuation, getInflectionLists, MySQLPipline, preprocessor

## Data Preprocessing

In [2]:
def country_replacement(s):
    try:
        al2 = pc.countries.get(alpha_2 = s).name
    except AttributeError:
        al2 = s
    return al2

In [3]:
class MySQLPipline:
    """ Define Class for MySQL Connection"""

    def __init__(self, database="funding"):
        """ Initialize object """
        self.conn = pymysql.connect(
            host='localhost', port=3306, user='root', passwd='Kyle9975', db=database, charset='utf8')
        self.conn.autocommit(True)
        self.cursor = self.conn.cursor()

    def NIHDataset(self):
        """ Processing the SQL query"""
        sql = """
                SELECT
                    *,
                    REGEXP_SUBSTR ( A.AwardNoticeDate, '[0-9]{4}' ) AS yr ,
                    REGEXP_SUBSTR ( A.ProjectStartDate, '[0-9]+/[0-9]+/[0-9]+' ) as sDate,
                    REGEXP_SUBSTR ( A.ProjectEndDate, '[0-9]+/[0-9]+/[0-9]+' ) as eDate
                FROM
                    (
                    SELECT DISTINCT
                        ProjectTitle,
                        ProjectNumber,
                        ContactPIProjectLeader,
                        ContactPIPersonID,
                        AwardNoticeDate,
                        ProjectStartDate,
                        ProjectEndDate,
                        OrganizationName,
                        ProjectAbstract,
                        OrganizationCity,
                        OrganizationState,
                        OrganizationCountry,
                        TotalCost
                    FROM
                        FundedDataNIH 
                    WHERE
                        ProjectAbstract IS NOT NULL 
                        AND ContactPIProjectLeader IS NOT NULL 
                    AND AwardNoticeDate IS NOT NULL 
                    ) A
                ORDER BY
                    --yr ASC
                """
        self.cursor.execute(sql)
        data = self.cursor.fetchall()
        data = pd.DataFrame(data)
        data.columns = ["title", "aID", "PILeader", "PIID", "aDate", "sDatetime", "eDateTime", "institution", "abstract", "city", "county", "country", "amount", "year", "sDate", "eDate"]
        return data

    def ERCDataset(self):
        """ Processing the SQL query"""
        sql = """
                SELECT DISTINCT
                    a.project_number,
                    a.project_ID,
                    SUBSTR(a.sDate, 7, 4) as `year`,
                    a.title,
                    b.Abstract,
                    b.Topic,
                    b.City,
                    b.Country,
                    b.HostInstitution,
                    a.amount,
                    a.sDate,
                    a.eDate 
                FROM
                    ERC_NEW a,
                    FundedDataERC b 
                WHERE
                    a.title = b.ProjectTitle
                    AND SUBSTR(a.sDate, 7, 4) > 2015
            """
        self.cursor.execute(sql)
        data = self.cursor.fetchall()
        data = pd.DataFrame(data)
        data.columns = ["aID", "callID",  "year", "title", "abstract", "topic", "county", "country",
                        "institution", "amount", "sDate", "eDate"]
        return data

    def NSFDataset(self):
        """ Processing the SQL query"""
        sql = """
                SELECT
                    AwardID,
                    AwardTitle,
                    Name_Institution,
                    AGENCY,
                    FirstName_Investigator,
                    LastName_Investigator,
                    SUBSTR(AwardEffectiveDate, 7, 4),
                    AwardEffectiveDate,
                    AwardExpirationDate,
                    AwardAmount,
                    AbstractNarration,
                    countryName_institution,
                    stateName_institution
                FROM
                    FundedDataNSF 
                WHERE
                    AbstractNarration IS NOT NULL
                    AND FirstName_Investigator IS NOT NULL
                    AND LastName_Investigator IS NOT NULL
                    AND AGENCY is not null
                    AND SUBSTR(AwardEffectiveDate, 7, 4) >= 2015
            """
        self.cursor.execute(sql)
        data = self.cursor.fetchall()
        data = pd.DataFrame(data)
        data.columns = ["aID", "title", "institution", "agency", "firstName", "lastName", "year", "sDate",
                        "eDate", "amount", "abstract", "country", "county"]
        return data

    def UKRIDataset(self):
        """ Processing the SQL query"""
        sql = """
                SELECT 
                        a.title,
                        a.proRef,
                        a.resSub,
                        a.resTopic,
                        a.orgName,
                        SUBSTR(a.sDate, 7, 4),
                        a.sDate,
                        a.eDate,
                        a.institution,
                        a.department,
                        a.projType,
                        a.PIFirstName,
                        a.PISurname,
                        a.Amount,
                        a.Abstract
                FROM
                        UKRI_Funded_ALL_Raw a
                WHERE
                        a.Abstract IS NOT NULL
                        AND a.PIFirstName IS NOT NULL
                        AND a.PISurname IS NOT NULL
                        AND SUBSTR(a.sDate, 7, 4) >= 2015
                """
        self.cursor.execute(sql)
        data = self.cursor.fetchall()
        data = pd.DataFrame(data)
        data.columns = ["title", "aID", "resSub", "resTopic", "orgName", "year", "sDate", "eDate", "institution",  "department", "projType", "PIFirName", "PISurname", "amount", "abstract"]
        return data
    
    def get_data(self, sql, colnames):
        self.cursor.execute(sql)
        data = self.cursor.fetchall()
        data = pd.DataFrame(data)
        data.columns = colnames
        return data

    def close_Conn(self):
        """ Closing Connection """
        self.cursor.close()
        self.conn.close()

### Importing Dataset

In [4]:
funding = MySQLPipline(database='funding')
NSFdata = funding.NSFDataset()
NSFdata["agency"] = ["NSF"] * NSFdata.shape[0]
NSFdata["sDate"] = pd.to_datetime(NSFdata['sDate'])
NSFdata["eDate"] = pd.to_datetime(NSFdata['eDate'])
NIHdata = funding.NIHDataset()
NIHdata["agency"] = ["NIH"] * NIHdata.shape[0]
NIHdata["sDate"] = pd.to_datetime(NIHdata['sDate'])
NIHdata["eDate"] = pd.to_datetime(NIHdata['eDate'])
ERCdata = funding.ERCDataset()
ERCdata["agency"] = ["ERC"] * ERCdata.shape[0]
ERCdata["sDate"] = pd.to_datetime(ERCdata['sDate'], dayfirst = True)
ERCdata["eDate"] = pd.to_datetime(ERCdata['eDate'], dayfirst = True)
UKRIdata = funding.UKRIDataset()
sql = "select region, ProjectReference from FundedDataUKRI"
UKRI_counties = funding.get_data(sql, colnames = ["county", "aID"])
UKRIdata = pd.merge(UKRIdata, UKRI_counties, how="inner", on=["aID"])
UKRIdata["agency"] = ["UKRI"] * UKRIdata.shape[0]
UKRIdata["country"] = ["UK"] * UKRIdata.shape[0]
UKRIdata["sDate"] = pd.to_datetime(UKRIdata['sDate'], dayfirst = True)
UKRIdata["eDate"] = pd.to_datetime(UKRIdata['eDate'], dayfirst = True)
funding.close_Conn()

### Combining datasets of all four agencies

In [5]:
df1 = pd.concat([ERCdata[["title", "abstract", "aID", "year", "amount", "institution", "country", "county", "agency", "sDate", "eDate"]], 
                 NIHdata[["title", "abstract", "aID", "year", "amount", "institution", "country", "county", "agency", "sDate", "eDate"]]])
df2 = pd.concat([NSFdata[["title", "abstract", "aID", "year", "amount", "institution", "country", "county", "agency", "sDate", "eDate"]],
                UKRIdata[["title", "abstract", "aID", "year", "amount", "institution", "country", "county", "agency", "sDate", "eDate"]]])
df = pd.concat([df1, df2])
df.reset_index(drop=True, inplace=True)
df = df[~pd.isna(df['abstract'])]
df.reset_index(drop=True, inplace=True)

### Handling different formats of country names

In [6]:
df.country = df.country.apply(lambda s: s.replace("ST KITTS/NEVIS", "Saint Kitts and Nevis").replace("TANZANIA U REP", "TANZANIA").replace("KOREA REP OF", "South Korea").replace("TRINIDAD/TOBA", "Trinidad and Tobago"))
df.country = df.country.apply(country_replacement)
df.country = df.country.apply(lambda s: " ".join([word.capitalize() for word in s.split()]))
df.country = df.country.apply(lambda s: 'UK' if s in ['Uk', 'United Kingdom'] else s)
df.country = df.country.apply(lambda s: 'US' if s in ["United States"] else s)
df.country = df.country.apply(lambda s: 'Greece' if s == 'El' else s)
df.country.unique()

array(['Poland', 'France', 'Finland', 'Denmark', 'Netherlands', 'Hungary',
       'Germany', 'Portugal', 'Greece', 'UK', 'Spain', 'Austria',
       'Romania', 'Israel', 'Turkey', 'Czechia', 'Italy', 'Bulgaria',
       'Sweden', 'Slovakia', 'Norway', 'US', 'Ireland', 'Switzerland',
       'Belgium', 'Serbia', 'Slovenia', 'Luxembourg', 'Croatia', 'Canada',
       'Cyprus', 'Australia', 'Estonia', 'Iceland', 'Lithuania',
       'New Zealand', 'Tunisia', 'Argentina', 'South Africa', 'Zimbabwe',
       'Ethiopia', 'Ghana', 'Peru', 'Thailand', 'Botswana', 'Tanzania',
       'Mozambique', 'Uganda', 'Nigeria', 'South Korea', 'Kenya',
       'Bangladesh', 'Suriname', 'Malawi', 'Colombia', 'Zambia', 'India',
       'Vietnam', 'Jamaica', 'Mali', 'Lebanon', 'Georgia', 'Pakistan',
       'Nepal', 'Rwanda', 'Guatemala', 'Trinidad And Tobago', 'Chile',
       'Nicaragua', 'Brazil', 'China', 'Sri Lanka', 'Haiti', 'Ukraine',
       'Japan', 'Mexico', 'Cameroon', 'Saint Kitts And Nevis', 'Uruguay',
    

### Preprocessing for topic modelling

In [7]:
pp = preprocessor(dataset = df)
# Removing unavailable projects
pp.remove_abstract(regex=r'Abstracts are not currently available in GtR').reset_index(drop = True, inplace = True)
pp.remove_abstract(regex=r'No abstract available').reset_index(drop = True, inplace = True)
pp.dataset = pp.dataset[~pd.isna(pp.dataset.title)]
pp.reset_index()

In [8]:
## Clean text
pp.clean_text(stemText=False)

This may take long time to run...
Starting to remove punctuations, numbers and multiple spaces...
Starting to remove projects with invalid abstracts...
Finished! It takes 51.46944189071655 seconds to run.


,title,abstract,aID,year,amount,institution,country,county,agency,sDate,eDate
0,Bioinspired Composites Strategies for Saving E...,Saving energy together with energy harvesting ...,948840,2021,1694375.0,AGH University of Science and Technology Krakow,Poland,Krakow,ERC,2021-01-01,2025-12-31
1,Pragmatics of Multiwinner Voting Algorithms a...,This proposal is in the area of computational ...,101002854,2021,1386290.0,AGH University of Science and Technology Krakow,Poland,Krakow,ERC,2021-06-01,2026-05-31
2,Exotic superconducting CIrcuits to Probe and p...,Quantum systems can occupy peculiar states suc...,851740,2020,1498875.0,ARMINES - Association for Research and Develop...,France,Paris,ERC,2020-03-01,2025-02-28
3,Quantum Feedback Engineering,Quantum technologies such as quantum computers...,884762,2020,2440125.0,ARMINES - Association for Research and Develop...,France,Paris,ERC,2020-12-01,2025-11-30
4,Artificial Enzyme Modules as Tools in a Tailor...,In order to tackle some of the prime societal ...,865885,2020,1995707.0,Aalto University,Finland,Aalto,ERC,2020-11-01,2025-10-31
...,...,...,...,...,...,...,...,...,...,...,...
145782,Mouse Models of Neurodegenerative Diseases Lab...,Neurodegeneration is defined by loss of specif...,MC_EX_MR/N501931/1,2017,1781285,MRC Mammalian Genetics Unit,UK,South East,UKRI,2017-01-01,2020-03-31
145783,SCREENswitch Switchable Opacity Patient Isola...,Many people have been put into isolation in ho...,79146,2020,59884,Korn Wall Limited,UK,Unknown,UKRI,2020-10-01,2020-12-31
145784,Virtual Cathode Deposition feasibility study f...,Increasing demand for small low cost powerful ...,132365,2016,104880,Plasma App Ltd.,UK,South East,UKRI,2016-07-01,2017-06-30
145785,Squidge amp Pip,Obtaining expert advice from a knowledgeable s...,752626,2015,5000,Lefendarty Limited,UK,Yorkshire and The Humber,UKRI,2015-08-01,2016-01-31


In [9]:
pp.reset_index()

In [10]:
## Checking duplicated data
pp.dataset[~pp.dataset[["title", "abstract", "agency"]].duplicated()]

,title,abstract,aID,year,amount,institution,country,county,agency,sDate,eDate
0,Bioinspired Composites Strategies for Saving E...,Saving energy together with energy harvesting ...,948840,2021,1694375.0,AGH University of Science and Technology Krakow,Poland,Krakow,ERC,2021-01-01,2025-12-31
1,Pragmatics of Multiwinner Voting Algorithms a...,This proposal is in the area of computational ...,101002854,2021,1386290.0,AGH University of Science and Technology Krakow,Poland,Krakow,ERC,2021-06-01,2026-05-31
2,Exotic superconducting CIrcuits to Probe and p...,Quantum systems can occupy peculiar states suc...,851740,2020,1498875.0,ARMINES - Association for Research and Develop...,France,Paris,ERC,2020-03-01,2025-02-28
3,Quantum Feedback Engineering,Quantum technologies such as quantum computers...,884762,2020,2440125.0,ARMINES - Association for Research and Develop...,France,Paris,ERC,2020-12-01,2025-11-30
4,Artificial Enzyme Modules as Tools in a Tailor...,In order to tackle some of the prime societal ...,865885,2020,1995707.0,Aalto University,Finland,Aalto,ERC,2020-11-01,2025-10-31
...,...,...,...,...,...,...,...,...,...,...,...
145782,Mouse Models of Neurodegenerative Diseases Lab...,Neurodegeneration is defined by loss of specif...,MC_EX_MR/N501931/1,2017,1781285,MRC Mammalian Genetics Unit,UK,South East,UKRI,2017-01-01,2020-03-31
145783,SCREENswitch Switchable Opacity Patient Isola...,Many people have been put into isolation in ho...,79146,2020,59884,Korn Wall Limited,UK,Unknown,UKRI,2020-10-01,2020-12-31
145784,Virtual Cathode Deposition feasibility study f...,Increasing demand for small low cost powerful ...,132365,2016,104880,Plasma App Ltd.,UK,South East,UKRI,2016-07-01,2017-06-30
145785,Squidge amp Pip,Obtaining expert advice from a knowledgeable s...,752626,2015,5000,Lefendarty Limited,UK,Yorkshire and The Humber,UKRI,2015-08-01,2016-01-31


In [11]:
df = pp.dataset

In [12]:
pp.dataset.to_csv("../Data/Cleaned_Data.csv", index = False)

### Getting the number of data observations for each agency

In [13]:
df.groupby("agency").agg({"abstract":'count', "title": 'count'})

,abstract,title
agency,,
ERC,5948,5948
NIH,71008,71008
NSF,41796,41796
UKRI,27035,27035


In [14]:
df.groupby("agency").agg({"abstract":'count', "title": 'count'})

,abstract,title
agency,,
ERC,5948,5948
NIH,71008,71008
NSF,41796,41796
UKRI,27035,27035
